In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Mon May 20 08:39:34 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   36C    P8               9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [ ]:
%%capture
!pip install datasets[audio]
!pip install evaluate
!pip install git+https://github.com/huggingface/transformers.git
!pip install jiwer
!pip install accelerate

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

## Prepare Data, Tokenizer, Feature Extractor

### Create `Wav2Vec2CTCTokenizer`

In [ ]:
from datasets import load_dataset, load_metric, Audio

df_train = load_dataset("google/fleurs", "sn_zw", split="train+validation")
df_test = load_dataset("google/fleurs", "sn_zw", split="test")

/usr/local/lib/python3.10/dist-packages/datasets/load.py:1486: FutureWarning: The repository for google/fleurs contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/google/fleurs
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(


Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

/usr/local/lib/python3.10/dist-packages/datasets/load.py:1486: FutureWarning: The repository for google/fleurs contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/google/fleurs
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(


In [ ]:
from datasets import ClassLabel
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)

    df = pd.DataFrame(dataset[picks])
    display(HTML(df.to_html()))

In [ ]:
show_random_elements(df_train.remove_columns(["path","id","raw_transcription","gender","lang_id","language","lang_group_id","num_samples","audio"]), num_examples=10)

,transcription
0,pashure pemazana emaawa ekushanda chinhu chinojeka mugirobhu racho chinopedzisira chatsva uye girobhu racho rinobva ramirawo kushanda
1,kune vakawanda vanoda kuzonzwa kuti zvava kuita kushanyira imwe nyeredzi kana kuti dzimwe nyika dziri muchadenga vamwe vanonetseka kuti kumbori nechii uye vamwe vanofunga kuti kunogara maalien kana kuti pane zvimwe zvinhu zvipenyu zvinogara ikoko
2,pfungwa imwe iri kunyanya kuitwa zvikuru kune avo vari kuronga kuti vangashandisa sei nguva yekumbozorora ndeye kushanya uye kudzidza
3,zvese gorofu nerugby zvirikutarisirwa kutambwa zvakare kumitambo yemaolympic
4,izvi zvisinei idambudziko rakakura kugadzirisa uye richatora makore mazhinji tisati taona mafusion reactor anoshandisika achizovakwa
5,up bow ine shepi yav uye down bow inenge yakafanana nesikweya ine mativi matatu asi isina rimwe divi rimwe chete
6,nguva kuyerawo kwatinoita kureba hurebu kwezvinoitika
7,kumaaustralian pfungwa yekofi yakangochena maitiro evekunze chitema chipfupi iespresso cappuccino inouya yakazadzwa pamusoro neruraza kwete furo zvakare tii inonwiwa isina mukaka
8,kune vaya vangada kubayira utefetefe nguva uye fungidziro yekuderedza mari yekushandisa kusvika pedyo nekusatombobhadhara chinhu ona bhajeti yemari shomanana pakushanya
9,zvakaziviswa kuti munhu wacho akafa nekenza yepanduru


Removing special characters

In [ ]:
import re
chars_to_remove_regex = '[\,\?\.\!\-\;\:\"\“\%\‘\”\�\']'

def remove_special_characters(batch):
    batch["transcription"] = re.sub(chars_to_remove_regex, '', batch["transcription"]).lower()
    return batch

In [ ]:
df_train = df_train.map(remove_special_characters)
df_test = df_test.map(remove_special_characters)

Map:   0%|          | 0/2856 [00:00<?, ? examples/s]

Map:   0%|          | 0/925 [00:00<?, ? examples/s]

Let's look at the processed text labels again.

In [ ]:
show_random_elements(df_train.remove_columns(["path","id","raw_transcription","gender","lang_id","language","lang_group_id","num_samples","audio"]))

,transcription
0,zviri nani kuti vanhu vanoda kusvika 100 vauya pamutambo kuti vapemberere mhemberero yekutanga yevaviri avo vaive nemuchato vakaitwa gore rakapera
1,rwizi rweamazon rwakakura makiromita 6 387 mamaira 3980 runodirwa mvura kubva kuzviuru zvenzizi diki
2,kana madzimai achishanyira nzvimbo dzinogara mamongi anotarisirwa kupfeka hembe dzakavhara mabvi nemapendekete
3,jas 39c gripen yakadonhera panhandare yendenge kunenge kuma930 dzemangwanani 0230 uct ikabva yaputika nhandare yacho ichibva yavharwa
4,mubatanidzwa une zvitsuwi 15 zvakapararira pamusoro pemamiriyoni 22 emakm2 egungwa
5,pataiteerera nyaya dzaitaurwa nemunhu mumwe chete mhuri kana kuti sangano takasvika pakuona zvaiitika kare uye kuziva vamwe vanhu vakabatsira kana kuti kukanganisa maitiro esangano racho
6,kakawanda mufambi akagadzikana uye ane chivimbo akakatyamadzwa nekufamba kunyika dziri kusimukira uko anofanira kuchinja maonero patsika nemagariro
7,tsvagurudzo yake yakaratidza kuti kana zvinoshanda mumunhu zvikapihwa zvaizomhanyisa kukura kwemapapu emwana ar mudumbu
8,mukati menguva dzino hondo inotyisa yakaitika pakati pevanoremekedzwa vakawanda vachirwira chigaro
9,chirwere chinotapurika chinokonzerwa nemapathogen akadai sevhairasi bhakitiriya netumwewo tupembenene


In [ ]:
def replace_hatted_characters(batch):
    batch["transcription"] = re.sub('[â]', 'a', batch["transcription"])
    batch["transcription"] = re.sub('[î]', 'i', batch["transcription"])
    batch["transcription"] = re.sub('[ô]', 'o', batch["transcription"])
    batch["transcription"] = re.sub('[û]', 'u', batch["transcription"])
    return batch

In [ ]:
df_train = df_train.map(replace_hatted_characters)
df_test = df_test.map(replace_hatted_characters)

Map:   0%|          | 0/2856 [00:00<?, ? examples/s]

Map:   0%|          | 0/925 [00:00<?, ? examples/s]

In [ ]:
def extract_all_chars(batch):
  all_text = " ".join(batch["transcription"])
  vocab = list(set(all_text))
  return {"vocab": [vocab], "all_text": [all_text]}

In [ ]:
vocab_train = df_train.map(extract_all_chars, batched=True, batch_size=-1, keep_in_memory=True, remove_columns=df_train.column_names)
vocab_test = df_test.map(extract_all_chars, batched=True, batch_size=-1, keep_in_memory=True, remove_columns=df_test.column_names)

Map:   0%|          | 0/2856 [00:00<?, ? examples/s]

Map:   0%|          | 0/925 [00:00<?, ? examples/s]

In [ ]:
vocab_list = list(set(vocab_train["vocab"][0]) | set(vocab_test["vocab"][0]))

In [ ]:
vocab_dict = {v: k for k, v in enumerate(sorted(vocab_list))}
vocab_dict

{' ': 0,
 '$': 1,
 '&': 2,
 '/': 3,
 '0': 4,
 '1': 5,
 '2': 6,
 '3': 7,
 '4': 8,
 '5': 9,
 '6': 10,
 '7': 11,
 '8': 12,
 '9': 13,
 '[': 14,
 ']': 15,
 '_': 16,
 'a': 17,
 'b': 18,
 'c': 19,
 'd': 20,
 'e': 21,
 'f': 22,
 'g': 23,
 'h': 24,
 'i': 25,
 'j': 26,
 'k': 27,
 'l': 28,
 'm': 29,
 'n': 30,
 'o': 31,
 'p': 32,
 'q': 33,
 'r': 34,
 's': 35,
 't': 36,
 'u': 37,
 'v': 38,
 'w': 39,
 'x': 40,
 'y': 41,
 'z': 42,
 '°': 43,
 '¾': 44,
 'á': 45,
 'ã': 46,
 'ç': 47,
 'é': 48,
 'ñ': 49,
 'ó': 50,
 'ö': 51,
 'ú': 52,
 'ü': 53}

In [ ]:
vocab_dict["|"] = vocab_dict[" "]
del vocab_dict[" "]

In [ ]:
vocab_dict["[UNK]"] = len(vocab_dict)
vocab_dict["[PAD]"] = len(vocab_dict)
len(vocab_dict)

56

In [ ]:
target_lang = "sn_zw"

Let's define an empty dictionary to which we can append the just created vocabulary

In [ ]:
new_vocab_dict = {target_lang: vocab_dict}

Let's now save the vocabulary as a json file.

In [ ]:
import json
with open('vocab.json', 'w') as vocab_file:
    json.dump(new_vocab_dict, vocab_file)

In a final step, we use the json file to load the vocabulary into an instance of the `Wav2Vec2CTCTokenizer` class.

In [ ]:
from transformers import Wav2Vec2CTCTokenizer

tokenizer = Wav2Vec2CTCTokenizer.from_pretrained("./", unk_token="[UNK]", pad_token="[PAD]", word_delimiter_token="|", target_lang=target_lang)

If one wants to re-use the just created tokenizer with the fine-tuned model of this notebook, it is strongly advised to upload the `tokenizer` to the Hub. Let's call the repo to which we will upload the files
`"wav2vec2-large-xlsr-turkish-demo-colab"`:

In [ ]:
repo_name = "ASR_fassy"

and upload the tokenizer

In [ ]:
tokenizer.push_to_hub(repo_name)

README.md:   0%|          | 0.00/5.17k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/fastinom/ASR_fassy/commit/736bea4c3c43bb2525f7e5d2b1d10439bdd13400', commit_message='Upload tokenizer', commit_description='', oid='736bea4c3c43bb2525f7e5d2b1d10439bdd13400', pr_url=None, pr_revision=None, pr_num=None)

### Create Wav2Vec2FeatureExtractor

In [ ]:
from transformers import Wav2Vec2FeatureExtractor

feature_extractor = Wav2Vec2FeatureExtractor(feature_size=1, sampling_rate=16000, padding_value=0.0, do_normalize=True, return_attention_mask=True)

In [ ]:
from transformers import Wav2Vec2Processor

processor = Wav2Vec2Processor(feature_extractor=feature_extractor, tokenizer=tokenizer)

Next, we can prepare the dataset.

In [ ]:
df_train[0]["transcription"]

'muna 1977 dr damadian vakapedza kugadzira mri scanner yaiona muviri wese iyo yavakati indomitable'

In [ ]:
df_train[0]["audio"]

{'path': 'train/10000813963377693741.wav',
 'array': array([0., 0., 0., ..., 0., 0., 0.]),
 'sampling_rate': 16000}

In [ ]:
df_train = df_train.cast_column("audio", Audio(sampling_rate=16_000))
df_test = df_test.cast_column("audio", Audio(sampling_rate=16_000))

Let's take a look at audio again.

In [ ]:
df_train[0]["audio"]

{'path': 'train/10000813963377693741.wav',
 'array': array([0., 0., 0., ..., 0., 0., 0.]),
 'sampling_rate': 16000}

This seemed to have worked! Let's listen to a couple of audio files to better understand the dataset and verify that the audio was correctly loaded.


In [ ]:
import IPython.display as ipd
import numpy as np
import random

rand_int = random.randint(0, len(df_train)-1)

print(df_train[rand_int]["transcription"])
ipd.Audio(data=df_train[rand_int]["audio"]["array"], autoplay=True, rate=16000)

It seems like the data is now correctly loaded and resampled.

In [ ]:
rand_int = random.randint(0, len(df_train)-1)

print("Target text:", df_train[rand_int]["transcription"])
print("Input array shape:", df_train[rand_int]["audio"]["array"].shape)
print("Sampling rate:", df_train[rand_int]["audio"]["sampling_rate"])

 the sampling rate always corresponds to 16kHz, and the target text is normalized.

In [ ]:
def prepare_dataset(batch):
    audio = batch["audio"]
    batch["input_values"] = processor(audio["array"], sampling_rate=audio["sampling_rate"]).input_values[0]
    batch["input_length"] = len(batch["input_values"])

    batch["labels"] = processor(text=batch["transcription"]).input_ids
    return batch

Let's apply the data preparation function to all examples.

In [ ]:
df_train = df_train.map(prepare_dataset, remove_columns=df_train.column_names)
df_test = df_test.map(prepare_dataset, remove_columns=df_test.column_names)

Map:   0%|          | 0/2856 [00:00<?, ? examples/s]

Map:   0%|          | 0/925 [00:00<?, ? examples/s]

start training

In [ ]:
import torch

from dataclasses import dataclass, field
from typing import Any, Dict, List, Optional, Union

@dataclass
class DataCollatorCTCWithPadding:
    """
    Data collator that will dynamically pad the inputs received.
    Args:
        processor (:class:`~transformers.Wav2Vec2Processor`)
            The processor used for proccessing the data.
        padding (:obj:`bool`, :obj:`str` or :class:`~transformers.tokenization_utils_base.PaddingStrategy`, `optional`, defaults to :obj:`True`):
            Select a strategy to pad the returned sequences (according to the model's padding side and padding index)
            among:
            * :obj:`True` or :obj:`'longest'`: Pad to the longest sequence in the batch (or no padding if only a single
              sequence if provided).
            * :obj:`'max_length'`: Pad to a maximum length specified with the argument :obj:`max_length` or to the
              maximum acceptable input length for the model if that argument is not provided.
            * :obj:`False` or :obj:`'do_not_pad'` (default): No padding (i.e., can output a batch with sequences of
              different lengths).
    """

    processor: Wav2Vec2Processor
    padding: Union[bool, str] = True

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lenghts and need
        # different padding methods
        input_features = [{"input_values": feature["input_values"]} for feature in features]
        label_features = [{"input_ids": feature["labels"]} for feature in features]

        batch = self.processor.pad(
            input_features,
            padding=self.padding,
            return_tensors="pt",
        )
        labels_batch = self.processor.pad(
            labels=label_features,
            padding=self.padding,
            return_tensors="pt",
        )

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        batch["labels"] = labels

        return batch

In [ ]:
data_collator = DataCollatorCTCWithPadding(processor=processor, padding=True)

Next, the evaluation metric is defined. WER

In [ ]:
from evaluate import load

wer_metric = load("wer")

In [ ]:
def compute_metrics(pred):
    pred_logits = pred.predictions
    pred_ids = np.argmax(pred_logits, axis=-1)

    pred.label_ids[pred.label_ids == -100] = processor.tokenizer.pad_token_id

    pred_str = processor.batch_decode(pred_ids)
    # we do not want to group tokens when computing the metrics
    label_str = processor.batch_decode(pred.label_ids, group_tokens=False)

    wer = wer_metric.compute(predictions=pred_str, references=label_str)

    return {"wer": wer}

In [ ]:
from transformers import Wav2Vec2ForCTC

model = Wav2Vec2ForCTC.from_pretrained(
    "facebook/mms-1b-all",
    attention_dropout=0.0,
    hidden_dropout=0.0,
    feat_proj_dropout=0.0,
    layerdrop=0.0,
    ctc_loss_reduction="mean",
    pad_token_id=processor.tokenizer.pad_token_id,
    vocab_size=len(processor.tokenizer),
    ignore_mismatched_sizes=True,
)

config.json:   0%|          | 0.00/2.04k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.86G [00:00<?, ?B/s]

Some weights of the model checkpoint at facebook/mms-1b-all were not used when initializing Wav2Vec2ForCTC: ['wav2vec2.encoder.pos_conv_embed.conv.weight_g', 'wav2vec2.encoder.pos_conv_embed.conv.weight_v']
- This IS expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/mms-1b-all and are newly initialized: ['wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original0', 'wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original1']
You should probably TRAIN this model on a down-stream

In [ ]:
model.init_adapter_layers()

Next, we freeze all weights, **but** the adapter layers.

In [ ]:
model.freeze_base_model()

adapter_weights = model._get_adapters()
for param in adapter_weights.values():
    param.requires_grad = True

In [ ]:
from transformers import TrainingArguments

training_args = TrainingArguments(
  output_dir=repo_name,
  group_by_length=True,
  per_device_train_batch_size=4,  # Reduced batch size
  evaluation_strategy="steps",
  num_train_epochs=3,  # Reduced number of epochs
  gradient_checkpointing=True,
  fp16=True,
  save_steps=400,  # Increased save steps
  eval_steps=200,  # Increased eval steps
  logging_steps=200,  # Increased logging steps
  learning_rate=5e-4,  # Slightly reduced learning rate
  warmup_steps=50,  # Reduced warmup steps
  save_total_limit=1,  # Reduced save limit
  push_to_hub=True,
)



/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Now, all instances can be passed to Trainer and we are ready to start training!

In [ ]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    data_collator=data_collator,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=df_train,
    eval_dataset=df_test,
    tokenizer=processor.feature_extractor,
)

### Training

In [ ]:
trainer.train()

/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:460: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss,Validation Loss,Wer
200,6.427400,4.151826,1.000000
400,3.797900,3.841069,1.000000
600,3.692400,3.424912,1.000000
800,0.835700,0.239659,0.267904
1000,0.152800,0.215555,0.248836
1200,0.141500,0.203627,0.243927
1400,0.127800,0.202865,0.240969


/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:460: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:460: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:460: UserWarning: torch.

In [ ]:
from safetensors.torch import save_file as safe_save_file
from transformers.models.wav2vec2.modeling_wav2vec2 import WAV2VEC2_ADAPTER_SAFE_FILE
import os

adapter_file = WAV2VEC2_ADAPTER_SAFE_FILE.format(target_lang)
adapter_file = os.path.join(training_args.output_dir, adapter_file)

safe_save_file(model._get_adapters(), adapter_file, metadata={"format": "pt"})